<br>

# Introdução

Juntar Código da CJ na tabela

In [ ]:
import html
import time

import pandas as pd
import requests
from bs4 import BeautifulSoup
from more_itertools import one
from open_geodata import geo
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from tjsp_divadmin.my_driver import Driver
from tjsp_divadmin.my_functions import (
    find_text_between_parenthesis,
    keep_numbers,
)
from tjsp_divadmin.my_paths import (
    adds_path,
    driver_path,
    logs_path,
    output_path_tab,
)
from traquitanas.scrapping import adds, gecko

In [ ]:
df_script1 = pd.read_csv(
    output_path_tab / 'Dim_Comarcas_CJs.csv'
)

df_script1.info()
df_script1.head()


In [ ]:
df_script2 = pd.read_csv(
    output_path_tab / 'Dim_Comarcas.csv'
)

df_script2.info()
df_script2.head()

<br>

Tento unir as tabelas para encontrar os erros!

In [ ]:
df_merged = pd.merge(
    left=df_script1,
    right=df_script2,
    left_on='comarca_nome_tjsp',
    right_on='comarca_nome_tjsp',
    how='left',    
)
df_merged

In [ ]:
# Coisas Erradas
df_merged[df_merged['comarca_nome'].isnull()]

<br>

Com a função abaixo eu atualizo os valores redigitos na tabela de maneira incorreta

In [ ]:
df_script1 = df_script1.replace(
    {        
        'Carapicuiba': 'Carapicuíba',
        'Cerqueira Cesar': 'Cerqueira César',
        'Estrela D’Oeste': 'Estrela dOeste',
        'Palmeira D’Oeste': "Palmeira d'Oeste",
        'Rio Grande de Serra': 'Rio Grande da Serra',
        'Santa Bárbara D’Oeste': "Santa Bárbara d'Oeste",
        'Santa Rosa do Viterbo': 'Santa Rosa de Viterbo',
        'Santana do Parnaíba': 'Santana de Parnaíba',
        'São Luis do Paraitinga': 'São Luiz do Paraitinga',
        #'Vila Mimosa': '' # Aqui estava o erro!!!!
    }
)

<br>

Rodo novamente, após correções

In [ ]:
df_merged = pd.merge(
    left=df_script1,
    right=df_script2,
    left_on='comarca_nome_tjsp',
    right_on='comarca_nome_tjsp',
    how='left',    
)
df_merged

<br>

Com essa função abaixo que fico procurando os erros!

In [ ]:
df_script2[df_script2['comarca_nome_tjsp'].str.startswith('Vila Mimosa')]

<br>

## Ajuste da Tabela

In [ ]:
# Coisas Certas
df_comarca = df_merged[~df_merged['comarca_nome'].isnull()]
df_comarca = df_comarca[['comarca_id', 'comarca_nome', 'comarca_nome_tjsp', 'cj_id']]
df_comarca['comarca_id'] = df_comarca['comarca_id'].astype('int')
df_comarca = df_comarca.iloc[df_comarca['comarca_nome'].str.normalize('NFKD').argsort()]
df_comarca= df_comarca.reset_index(drop=True)

df_comarca.info()
df_comarca.head()

In [ ]:
# Salva
df_comarca.to_csv(
    output_path_tab / 'Dim_Comarcas.csv',
    index=False,
)
df_comarca.to_excel(
    output_path_tab / 'Dim_Comarcas.xlsx',
    sheet_name='Comarcas',
    index=False,
)


<br>

## Deleta Arquivos Errados

Gerados pelo script 01, que raspou o "Quem Somos"

In [ ]:
for file in list(output_path_tab.glob('Dim_Comarcas_CJs*')):
    print(file)
    file.unlink()